In [1]:
import warnings
warnings.filterwarnings(action='ignore') 


In [2]:
import math
import time
import pandas as pd

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gc


In [3]:
from dataloader import makeDatapathList,dataTransform,VOCDataset

root_path="./data/VOCdevkit/VOC2012/"

datapath_list=makeDatapathList(root_path)
train_img_list,train_anno_list=datapath_list('train')
val_img_list,val_anno_list=datapath_list('val')

color_mean=(0.485,0.456,0.406)
color_std=(0.29,0.224,0.225)

transform=dataTransform(475,color_mean,color_std)
train_dataset=VOCDataset(train_img_list,train_anno_list,phase="train",transform=transform)
val_dataset=VOCDataset(val_img_list,val_anno_list,phase="val",transform=transform)

batch_size=4
train_dataloader=data.DataLoader(train_dataset,batch_size,True)
val_dataloader=data.DataLoader(val_dataset,batch_size,False)

dataloaders_dict={"train":train_dataloader,"val":val_dataloader}

In [4]:
from pspnet import PSPNet

net=PSPNet(n_classes=150)

state_dict=torch.load("./weights/pspnet50_ADE20K.pth") #fine tuning
net.load_state_dict(state_dict, strict=False)

n_classes=21
net.decode_feature.classification=nn.Conv2d(in_channels=512,out_channels=n_classes,kernel_size=1,stride=1,padding=0)
net.aux.classification=nn.Conv2d(in_channels=256,out_channels=n_classes,kernel_size=1,stride=1,padding=0)

def weights_init(m):
    if isinstance(m,nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias,0.0)

net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)


Conv2d(256, 21, kernel_size=(1, 1), stride=(1, 1))

In [5]:
class PSPLoss(nn.Module):
    def __init__(self,aux_weight=0.4):
        super(PSPLoss,self).__init__()
        self.aux_weight=aux_weight
    
    def forward(self,outputs,targets):
        loss=F.cross_entropy(outputs[0],targets,reduction='mean')
        loss_aux=F.cross_entropy(outputs[1],targets,reduction='mean')
        return loss+self.aux_weight*loss_aux

In [6]:
criterion=PSPLoss()

In [7]:
optimizer=optim.SGD([
    {'params':net.feature_conv.parameters(),'lr':1e-3},
    {'params':net.feature_res_1.parameters(),'lr':1e-3},
    {'params':net.feature_res_2.parameters(),'lr':1e-3},
    {'params':net.feature_dilated_res_1.parameters(),'lr':1e-3},
    {'params':net.feature_dilated_res_2.parameters(),'lr':1e-3},
    {'params':net.pyramid_pooling.parameters(),'lr':1e-3},
    {'params':net.decode_feature.parameters(),'lr':1e-3},
    {'params':net.aux.parameters(),'lr':1e-3},
],momentum=0.9,weight_decay=0.0001)

def lambda_epoch(epoch):
    max_epoch=30
    return math.pow((1-epoch/max_epoch),0.9)

scheduler=optim.lr_scheduler.LambdaLR(optimizer,lr_lambda=lambda_epoch)

In [8]:
def train_model(net,dataloaders_dict,criterion,scheduler,optimizer,num_epochs):
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    net.to(device)
    torch.backends.cudnn.benchmark=True

    num_train_imgs=len(dataloaders_dict["train"].dataset)
    num_val_imgs=len(dataloaders_dict["val"].dataset)
    
    batch_size=dataloaders_dict["train"].batch_size

    iteration=1
    logs=[]

    batch_multiplier=3

    for epoch in range(num_epochs):
        gc.collect()
        torch.cuda.empty_cache()
        t_epoch_start=time.time()
        t_iter_start=time.time()
        epoch_train_loss=0.0
        epoch_val_loss=0.0

        print(f"{'='*5}Epoch {epoch+1}/{num_epochs}{'='*5}")
        for phase in ["train","val"]:
            if phase=="train":
                net.train()
                scheduler.step()
                optimizer.zero_grad()
                print("[train]")
            else:
                if (epoch+1)%5==0:
                    net.eval()
                    print("[val]")
                else: 
                    continue
            count=0
            for imgs,anno_class_imgs in dataloaders_dict[phase]:
                if imgs.size()[0]==1:
                    continue
                imgs=imgs.to(device)
                anno_class_imgs=anno_class_imgs.to(device)

                if phase=='train' and count==0:
                    optimizer.step()
                    optimizer.zero_grad()
                    count=batch_multiplier
                with torch.set_grad_enabled(phase=="train"):
                    outputs=net(imgs)
                    loss=criterion(outputs,anno_class_imgs.long())/batch_multiplier
                    if phase=="train":
                        loss.backward()
                        count-=1
                        if iteration%10==0:
                            t_iter_finish=time.time()
                            print("iter [{}] loss:{:.4f}   | {:.4f}sec".format(iteration,loss.item()/batch_size*batch_multiplier,t_iter_finish-t_iter_start))
                            t_iter_start=time.time()
                        epoch_train_loss+=loss.item()*batch_multiplier
                        iteration+=1
                    else:
                        epoch_val_loss+=loss.item()*batch_multiplier
            t_epoch_finish=time.time()
            print("epoch [{}] train loss:{:.4f}, val loss:{:.4f}  | {:.4f}sec".format(epoch+1,epoch_train_loss/num_train_imgs,epoch_val_loss/num_val_imgs,t_epoch_finish-t_epoch_start))
            t_epoch_start=time.time()

            log_epoch={'epoch':epoch+1,'train_loss':epoch_train_loss/num_train_imgs,'val_loss':epoch_val_loss/num_val_imgs}
            logs.append(log_epoch)
            df=pd.DataFrame(logs)
            df.to_csv('log_output.csv')
        torch.save(net.state_dict(),'weights/pspnet50_'+str(epoch+1)+'.pth')

In [9]:
num_epochs=30
train_model(net,dataloaders_dict,criterion,scheduler,optimizer,num_epochs)

cuda:0
=====Epoch 1/30=====
[train]
iter [10] loss:1.3323   | 5.9815sec
iter [20] loss:1.1108   | 3.1522sec
iter [30] loss:0.7835   | 3.1728sec
iter [40] loss:0.7306   | 3.1578sec
iter [50] loss:0.3389   | 3.2765sec
iter [60] loss:0.9328   | 3.4631sec
iter [70] loss:0.5702   | 3.3025sec
iter [80] loss:0.3518   | 3.2564sec
iter [90] loss:0.2815   | 3.1743sec
iter [100] loss:0.3610   | 3.1911sec
iter [110] loss:0.5245   | 3.2129sec
iter [120] loss:0.6376   | 3.2044sec
iter [130] loss:0.3975   | 3.1892sec
iter [140] loss:0.3770   | 3.0868sec
iter [150] loss:0.3989   | 3.1769sec
iter [160] loss:0.3410   | 3.0710sec
iter [170] loss:0.2409   | 2.9535sec
iter [180] loss:0.4378   | 3.0617sec
iter [190] loss:0.4378   | 3.1677sec
iter [200] loss:0.2121   | 3.0639sec
iter [210] loss:0.6483   | 2.9577sec
iter [220] loss:0.3859   | 2.9261sec
iter [230] loss:0.1193   | 2.9271sec
iter [240] loss:0.4662   | 2.9602sec
iter [250] loss:0.2357   | 2.9338sec
iter [260] loss:0.1840   | 2.9438sec
iter [270] 